# Archivo Historico de Localidades
(Historical Archive of Localities)

In [33]:
from zipfile import ZipFile
from io import BytesIO
import pandas as pd
import numpy as np


In [34]:
file_name = 'C:/Users/Lorrain/Documents/Datos/AHL00.zip' #Archivo principal

Contenido del archivo principal / Content of main archive:

In [35]:
with ZipFile(file_name, 'r') as zip_file:
    print('Contenido del zip:', end = '\n\n')
    lista_zips = zip_file.namelist()
    print(lista_zips)

Contenido del zip:

['AHL01.zip', 'AHL02.zip', 'AHL03.zip', 'AHL04.zip', 'AHL05.zip', 'AHL06.zip', 'AHL07.zip', 'AHL08.zip', 'AHL09.zip', 'AHL10.zip', 'AHL11.zip', 'AHL12.zip', 'AHL13.zip', 'AHL14.zip', 'AHL15.zip', 'AHL16.zip', 'AHL17.zip', 'AHL18.zip', 'AHL19.zip', 'AHL20.zip', 'AHL21.zip', 'AHL22.zip', 'AHL23.zip', 'AHL24.zip', 'AHL25.zip', 'AHL26.zip', 'AHL27.zip', 'AHL28.zip', 'AHL29.zip', 'AHL30.zip', 'AHL31.zip', 'AHL32.zip']


## Archivo habitantes / Habitantes file

En general INEGI utiliza la codificacion 'utf-8', pero hay veces que los datos estan en 'Windows-1252' (tambien con codigo 'cp1252') cuando los manipula y exporta desde Excel (o tal vez Access).  

El archivo zip no es tan grande y se tiene la suficente memoria para extraer los archivos sin descomprimir. 

--------------------------------------------

INEGI normally encodes its data in 'utf-8', but sometimes it is in 'Windows-1252' (which is the same as 'cp1252') when its manipulated and exported from Excel (or maybe Access). 

The zipfile is not that big and the computer has enough memory so files will be extracted without uncompressing.

In [39]:
with ZipFile(file_name, 'r') as zip_file: # Primer zip
    sub_zip = BytesIO(zip_file.read(lista_zips[0])) # Segundo zip
    with ZipFile(sub_zip, 'r') as info_estatal:
        lista_estatal = info_estatal.namelist()
        
        print('Contenido del zip estatal / State zip content:', end = '\n\n')
        print(lista_estatal, end = '\n\n')
        
        print('Guardando informacion en variable: - habitantes -')
#\n\n\t- maestro - movimientos - res_hist -')
        
        with info_estatal.open('AHL01Habitantes.csv', 'r') as archivo: 
            habitantes = pd.read_csv(archivo, encoding = 'cp1252', 
                                     dtype={
                                         'CVE_GEOEST': str # Definir la columna como string
                                     })


Contenido del zip estatal / State zip content:

['AHL01Habitantes.csv', 'AHL01Maestro.csv', 'AHL01Movimientos.csv', 'AHL01Res_hist.csv', 'Estructura_Archivos_AHL.pdf']

Guardando informacion en variable: - habitantes -


In [28]:
habitantes.head()

,CLAVE,CVE_GEOEST,EVE_CENSAL,INDICE_HAB,FUENTE,TOT_HAB,TOT_HOM,TOT_MUJ
0,1,010010001,1900,NaN,Censo,35052,16229,18823
1,1,010010001,1910,NaN,Censo,45198,20979,24219
2,1,010010001,1921,NaN,Censo,48041,21690,26351
3,1,010010001,1930,NaN,Censo,62244,28687,33557
4,1,010010001,1940,NaN,Censo,82234,37821,44413


Las claves en INEGI deben de ser manejadas como texto. Esto es cierto para CVE_GEOEST, que representa la clave geoestadistica de una localidad especifica. Si se importa como numero entonces a clave '010010001' se importará como 10010001. En otras palabras se eliminará el cero al principio de la clave que es necesario porque representa al estado en el que se encuentra la localidad y hay 32 en el pais.

Cada localidad tiene una clave que esta formada por 9 caracteres numericos. Los primeros dos representan la clave del estado ('01'), los siguientes tres caracteres representan al municipio ('001'), y los ultimos cuatro numeros representan la clave de la localidad ('0001'). Los Estados estan organizados por orden alfabetico. Los municipios y localidades no son consistentes con este criterio porque cuando se crean nuevos la numeración no es reiniciada. Solo hay una regla de nomenclatura en las localidades, la clave '0001' siempre representará la cabecera municipal.


--------------------------------------------

Keys in INEGI should be treated like strings. This is true ofre CVE_GEOEST that representas the geo-statistical key of a specific locality. If this field is imported like a number then the key '010010001' will be imported with the format 10010001. It will eliminate the zero at hte begining an it is necesario because it represents the state in with de locality is in and there are 32 in the country.

Each locality has a key that is formed by 9 numerical caracters. The first two represent the state key ('01'), the next three represent the municipality ('001'), and the las four numberes represent the key of the locality ('0001'). The States are organized in alphabetical order. The municipalites and localities are not consistent with this criteria because when a new entitty is created the numeration is no reiniciated.There is only one nomenclature rull in localities, the key '0001' will always represent the municipal head (the locality where the municipal government resides).

In [73]:
habitantes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18393 entries, 0 to 18392
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CLAVE       18393 non-null  int64  
 1   CVE_GEOEST  18393 non-null  object 
 2   EVE_CENSAL  18393 non-null  int64  
 3   INDICE_HAB  0 non-null      float64
 4   FUENTE      18393 non-null  object 
 5   TOT_HAB     17109 non-null  float64
 6   TOT_HOM     11691 non-null  float64
 7   TOT_MUJ     11677 non-null  float64
dtypes: float64(4), int64(2), object(2)
memory usage: 1.1+ MB


La columna CLAVE es un numero consecutivo que es diferente para cada localidad del estado. Por eso tiene la misma cantidad de valores unicos que CVE_GEOEST.

EVE_CENSAL representa el evento censal donde se obuvieron las mediciones de las columnas TOT_HAB (Total de habitantes), TOT_HOM (Total de hombres), y TOT_MUJ (Total de mujeres).

Los valores unicos de FUENTE representan el tipo de evento censal.

--------------------------------

CLAVE is a consecutive number that is diferent for each locality of the state. That is why it has the same cuantity of values as CVE_GEOEST.

EVE_CENSAL represents the censal event where the data of TOT_HAB (Total inhabitants), TOT_HOM (Men total), and TOT_MUJ (Women total).

The unique values of FUENTE represent the tipe of censal event (Count or full Cense).

In [57]:
lista_columnas = habitantes.columns.values.tolist()

for col in lista_columnas[ :5]: 
    print(f'Cantidad de valores unicos de {col}: {len(habitantes[col].unique())}')
    
print()
print('Valores unicos de EVE_CENSAL:', end = '\n\n')
print(habitantes['EVE_CENSAL'].unique(), end = '\n\n')

print('Valores unicos de FUENTE:', end = '\n\n')
print(habitantes['FUENTE'].unique(), end = '\n\n')

print('Valores unicos de INDICE_HAB:', end = '\n\n')
print(habitantes['INDICE_HAB'].unique(), end = '\n\n')



Cantidad de valores unicos de CLAVE: 3903
Cantidad de valores unicos de CVE_GEOEST: 3903
Cantidad de valores unicos de EVE_CENSAL: 15
Cantidad de valores unicos de INDICE_HAB: 1
Cantidad de valores unicos de FUENTE: 2

Valores unicos de EVE_CENSAL:

[1900 1910 1921 1930 1940 1950 1960 1970 1980 1990 1995 2000 2005 2010
 2020]

Valores unicos de FUENTE:

['Censo' 'Conteo']

Valores unicos de INDICE_HAB:

[nan]



In [70]:
habitantes['TOT_HAB'] =pd.to_numeric(habitantes['TOT_HAB'], 
                                     errors='coerce')

habitantes['TOT_HOM'] =pd.to_numeric(habitantes['TOT_HOM'], 
                                     errors='coerce',
                                     downcast='integer')

habitantes['TOT_MUJ'] =pd.to_numeric(habitantes['TOT_MUJ'], 
                                     errors='coerce',
                                     downcast='integer')

In [72]:
habitantes.loc[habitantes['EVE_CENSAL'] == 1921]

,CLAVE,CVE_GEOEST,EVE_CENSAL,INDICE_HAB,FUENTE,TOT_HAB,TOT_HOM,TOT_MUJ
2,1,010010001,1921,NaN,Censo,48041.0,21690.0,26351.0
38,6,010010097,1921,NaN,Censo,124.0,73.0,51.0
76,12,010010104,1921,NaN,Censo,37.0,24.0,13.0
91,13,010010105,1921,NaN,Censo,26.0,12.0,14.0
113,15,010010107,1921,NaN,Censo,57.0,32.0,25.0
...,...,...,...,...,...,...,...,...
16492,3231,010090028,1921,NaN,Censo,16.0,6.0,10.0
16545,3238,010090037,1921,NaN,Censo,62.0,33.0,29.0
16613,3251,010090051,1921,NaN,Censo,154.0,78.0,76.0
16656,3259,010090061,1921,NaN,Censo,149.0,78.0,71.0
